In [11]:
%pip install youtube-transcript-api tiktoken faiss-cpu

Note: you may need to restart the kernel to use updated packages.


# Indexing
## Step 1 (Document Ingestion)

In [12]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [17]:
youtube_id="W4tqbEmplug"
transcript_list=YouTubeTranscriptApi.get_transcript(youtube_id,languages=["en"])

In [18]:
transcript=(" ").join(chunk['text'] for chunk in transcript_list)
transcript

'Let\'s say you have a fight with your girlfriend. You want to do the right thing. So you go to chat GBT and you be like, "This is exactly what happened. Tell me what to do." And you go, "Babe, I just want you to know I want to take full accountability and I care about this relationship." And she says, "Did you get this answer from chat GBT?" And you go, "I did." How\'s that going to go now? You did everything right, but what makes people beautiful is not that we get everything right. It\'s that we get many things wrong. And I think in the modern world we live in, we forget that. Simon Synynic is back. He\'s the visionary thinker inspiring millions to cultivate human connection, find their purpose, and overcome any modern-day challenges. So with AI, do you think it really is cause for concern and deeper thought? So I\'m not in the AI business, but I am in the humanity business. And here\'s the problem that we never talk about here. People keep telling us life is not about the destinati

### Indexing step 2 (Text Splitting)

In [22]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks=splitter.create_documents([transcript])

In [25]:
chunks[101].page_content

"we sat around and you know who some of these people are you know and we sat around a room and they spent all of their time talking about how they can share their lists with each other and how they can cross-promote with each other and what margins are you getting on this? And I'm like, you guys are and I spoke up. I was like, you guys are just talking about making more money. What? I thought we came here to like do common good together. They didn't invite me back. But it was but the point is is like the money and the fame is seductive. You know this is this is this is this is Joseph Campbell's hero's journey which is you start off as the reluctant hero and you get called to your mission for some reason and at some point in your journey there will be something seduce you away from your mission money or power you know Luke Skywalker was nearly seduced away from the vision you know from the journey like this is what this is what the hero has to go through and do you have the friends who"

### Embedding generation and Vector Store Creation

In [26]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()
vector_store=FAISS.from_documents(chunks,embedding)

In [28]:
vector_store.index_to_docstore_id

{0: '3ed007f2-e265-4ab5-9a02-48f99578720c',
 1: '86a7567d-a218-4128-b71a-6447bd891f97',
 2: 'e3679a10-ec6e-42bf-a448-33f91c227785',
 3: '50b2a266-37c7-4fee-8add-ca5a4fb2a68e',
 4: '2cc1e597-279b-4d65-810c-e75a506a40f9',
 5: 'c299698c-a976-4b37-88b0-be82f9251507',
 6: '7047123c-d116-469c-b1b3-52c1e75886cc',
 7: 'f310c6b7-ed77-4808-b128-db2c513fa03d',
 8: '92a8944a-fc49-4e57-b6ed-7722a95b5c43',
 9: 'fc20e718-15aa-4657-b0c3-ad5b51a278ac',
 10: 'a9685e06-26a9-4062-9388-7441541e9ef2',
 11: 'f981c50b-078b-4e40-8465-18be4222629c',
 12: 'c4a99f39-5209-4995-baac-6701d5128c91',
 13: 'b1f43eae-2427-46fd-bf4a-8f5ba02e121d',
 14: '066035d1-1a21-43ac-bca6-47cdf8873cb5',
 15: 'd64f4cca-8df8-47c8-8122-903ec863c2ee',
 16: 'efc2f82f-23e6-4bad-afd0-5e2977341e26',
 17: '36a35fbf-5654-4d8a-a982-28ade6eb1cda',
 18: 'f1a125ab-4d7e-4cc0-82c8-31e01485ba66',
 19: 'f8d2f895-30ec-4417-8210-9feaa4c0e191',
 20: 'c5712adc-3857-48e5-aca8-e09b9845f2c8',
 21: '66ce2377-45f4-4ed1-bd8b-4b8e5a4f1978',
 22: 'cd7df22c-3235-

In [33]:
ex=vector_store.get_by_ids(['655cb2c3-c31c-414c-b9f4-64fed7af8fc5'])

In [ ]:
print(ex[0].page_content)    

for the work that I did and it had my words on it. So it's not my words, it's their words. I just helped distill them. It's who they are when they're at their natural best. I just helped them put it and codify it. Simon, thank you. Thanks. Always love our conversations and long may they continue. Likewise. This has always blown my mind a little bit. 53% of you that listen to this show regularly haven't yet subscribed to the show. So, could I ask you for a favor? If you like the show and you like what we do here and you want to support us, the free simple way that you can do just that is by hitting the subscribe button. And my commitment to you is if you do that, then I'll do everything in my power, me and my team, to make sure that this show is better for you every single week. We'll listen to your feedback. We'll find the guests that you want me to speak to and we'll continue to do what we do. Thank you so much. Heat. Heat. N. [Music] [Music]


# **Step 2 Retrieval**

In [38]:
retiever=vector_store.as_retriever(search_type="similarity",search_kwargs={"k":4})

In [39]:
docs=retiever.invoke("What is AI")

In [43]:
docs

[Document(id='27028bb3-4d2b-4411-8106-93352d11d48e', metadata={}, page_content="that make us better what we do better better than how we show up in the world. And AI will absolutely make our lives easier. Like most technology makes our lives easier. That's kind of the role, the rule of technology, which is to make life a little easier, a little more efficient, a little quicker, a little less, you know, strain on the muscles. you know, that's kind of what it does, you know, from the from the plow, you know, all the way up to the internet and AI. It just makes life a little easier. But we're still human beings who are forced to live with human beings. You're writing a book about friendship that we're all waiting for. Yeah. I'm not going to ask you when it's due cuz I don't want to be your publisher, but um I know that they they chase and chase and chase and chase, but my I guess my closing question is why? Of all the things you could have written about, Simon, you're someone who's able t

In [44]:
docs[0].page_content

"that make us better what we do better better than how we show up in the world. And AI will absolutely make our lives easier. Like most technology makes our lives easier. That's kind of the role, the rule of technology, which is to make life a little easier, a little more efficient, a little quicker, a little less, you know, strain on the muscles. you know, that's kind of what it does, you know, from the from the plow, you know, all the way up to the internet and AI. It just makes life a little easier. But we're still human beings who are forced to live with human beings. You're writing a book about friendship that we're all waiting for. Yeah. I'm not going to ask you when it's due cuz I don't want to be your publisher, but um I know that they they chase and chase and chase and chase, but my I guess my closing question is why? Of all the things you could have written about, Simon, you're someone who's able to traverse se several subject matters across business and life and everything"

In [45]:
context="\n\n".join(doc.page_content for doc in docs)
context

'that make us better what we do better better than how we show up in the world. And AI will absolutely make our lives easier. Like most technology makes our lives easier. That\'s kind of the role, the rule of technology, which is to make life a little easier, a little more efficient, a little quicker, a little less, you know, strain on the muscles. you know, that\'s kind of what it does, you know, from the from the plow, you know, all the way up to the internet and AI. It just makes life a little easier. But we\'re still human beings who are forced to live with human beings. You\'re writing a book about friendship that we\'re all waiting for. Yeah. I\'m not going to ask you when it\'s due cuz I don\'t want to be your publisher, but um I know that they they chase and chase and chase and chase, but my I guess my closing question is why? Of all the things you could have written about, Simon, you\'re someone who\'s able to traverse se several subject matters across business and life and ev

### Prompt Template

In [46]:
template=PromptTemplate(
    template="""Act as a helpful assistant. You have to give answer of question from the following context.
    {context}
    Question: {query}
    If the context is insufficient to answer. Just say you don't know.
    """,
    input_variables=["context","query"]
)

### Augmentation

In [47]:
query="What is the reason that AI is discussed in this?"
ret_docs=retiever.invoke(query)

In [48]:
ret_context="\n\n".join(doc.page_content for doc in ret_docs)

In [49]:
prompt=template.invoke({
    "context":ret_context,
    "query":query
})

In [50]:
prompt

StringPromptValue(text='Act as a helpful assistant. You have to give answer of question from the following context.\n    that make us better what we do better better than how we show up in the world. And AI will absolutely make our lives easier. Like most technology makes our lives easier. That\'s kind of the role, the rule of technology, which is to make life a little easier, a little more efficient, a little quicker, a little less, you know, strain on the muscles. you know, that\'s kind of what it does, you know, from the from the plow, you know, all the way up to the internet and AI. It just makes life a little easier. But we\'re still human beings who are forced to live with human beings. You\'re writing a book about friendship that we\'re all waiting for. Yeah. I\'m not going to ask you when it\'s due cuz I don\'t want to be your publisher, but um I know that they they chase and chase and chase and chase, but my I guess my closing question is why? Of all the things you could have 

### Generation

In [51]:
llm= ChatOpenAI()
response=llm.invoke(prompt)
response.content

'The reason AI is discussed in this context is because it is seen as a technology that has the potential to make our lives easier and more efficient. There is a concern raised about the impact of AI on job displacement and the value of personal growth through struggle and learning. It is also mentioned that AI is changing the way we work and live, and there are worries about winners and losers in the advancement of AI technology.'